In [1]:
## 导入需要的库
import numpy as np
import pandas as pd
import math
import os
import random
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

In [2]:
## 读取数据
df = pd.read_excel("2022年上海证券交易所50指数（上证50指数）数据集/预处理后的数据.xlsx")

In [3]:
df.head()

,股票代码,8月1日,8月2日,8月3日,8月4日,8月5日,8月8日,8月9日,8月10日,8月11日,...,12月12日,12月13日,12月14日,12月15日,12月16日,12月19日,12月20日,12月21日,12月22日,12月23日
0,601857.SHA,7.701332,7.553797,7.494783,7.480030,7.450522,7.465276,7.524290,7.568551,7.657072,...,7.801168,7.816495,7.877801,7.801168,7.831821,7.693883,7.663230,7.647903,7.663230,7.632577
1,601668.SHA,10.353531,10.167540,10.002213,10.043545,10.146873,10.105542,10.022879,9.960882,10.084876,...,11.924127,11.924127,11.800132,11.593474,11.758800,11.428148,11.345486,11.283488,11.221492,11.200826
2,600010.SHA,18.395260,17.639290,17.807283,17.723288,17.975277,17.975277,18.143270,18.059273,18.227266,...,16.547335,17.135311,16.799324,16.631330,16.631330,16.211348,16.211348,16.043354,15.791365,15.875361
3,600028.SHA,15.531209,15.417843,15.342265,15.380054,15.417843,15.493421,15.531209,15.531209,15.644576,...,17.583673,17.583673,17.505175,17.348177,17.465925,17.073433,17.034184,17.112682,17.112682,17.034184
4,601919.SHA,19.505402,18.497742,18.512136,18.670485,19.030361,19.433426,19.476610,19.303870,19.562983,...,18.886410,18.670485,18.721586,18.363848,18.977112,18.500130,18.278673,18.210533,18.346813,18.210533


In [4]:
#根据日收盘价计算日收益率，再计算日对数收益率
df1 = np.zeros((len(df),len(df.T)-2))  #日收益率
df2 = np.zeros((len(df),len(df.T)-2))  #日对数收益率
for i in range(len(df)):
    for j in range(len(df.T)-2):
        df1[i][j] = (df.loc[i][j+2]-df.loc[i][j+1])/(df.loc[i][j+1])
        df2[i][j] = math.log(1+df1[i][j])
df2

array([[-0.01934292, -0.0078432 , -0.00197043, ..., -0.00200202,
         0.00200202, -0.00400803],
       [-0.01812733, -0.01639387,  0.00412372, ..., -0.00547954,
        -0.00550955, -0.00184334],
       [-0.04196422,  0.00947873, -0.00472805, ..., -0.01041681,
        -0.01583141,  0.00530501],
       ...,
       [-0.01518162, -0.00397392,  0.01187463, ...,  0.0139999 ,
        -0.00258983,  0.0080711 ],
       [-0.01312341, -0.00247214, -0.00136237, ...,  0.00267464,
         0.0189621 , -0.03060443],
       [-0.00547445,  0.00266675,  0.0163171 , ...,  0.0138972 ,
         0.0165387 ,  0.00169538]])

In [6]:
##根据算法对数据进行聚类
#归一化数据
n, m = len(df2), len(df2.T)
data1 = np.zeros((n,m))
for i in range(n):
    a = min(df2[i])
    b = max(df2[i])
    
    for j in range(m):
        data1[i][j]=(df2[i][j]-a)/(b-a)
data2 = pd.DataFrame(data1)
data2.to_csv(r"上海预处理后的数据归一化数据.csv")

In [7]:
def GYjuli(i,j):
    theta = 0.5
    m = len(df2.T)
    n = len(df2)
    a, b, a1, b1 = [], [], [], []
    X1 = np.zeros((1,m))
    X2 = np.zeros((1,m))
    A, B, A1, B1, D = 0, 0, 0, 0, 0
    # 取最大最小值
    for l in range(n):
        a.append(min(abs(data2.loc[i]-data2.loc[l])))
        b.append(max(abs(data2.loc[i]-data2.loc[l])))
        a1.append(min(abs(data2.loc[j]-data2.loc[l])))
        b1.append(max(abs(data2.loc[j]-data2.loc[l])))
    
    A, B = min(a), max(b)
    A1, B1 = min(a1), max(b1)
    c = abs(data2.loc[i]-data2.loc[j])
    
    for k in range(m):
        X1[0][k] = (A+theta*B)/(c[k]+theta*B) # 计算不同属性下的灰色关联系数
        X2[0][k] = (A1+theta*B1)/(c[k]+theta*B1)
    # print(X1)
    # print(X2)
    # D = np.sqrt(np.sum(np.square(X1 - X2))) #欧式距离
    D = np.sum(abs(X1 - X2)) #汉明距离
    return D

In [8]:
D = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if i<j: 
            D[i][j] = GYjuli(i,j)
        if i>j:
            D[i][j] = D[j][i]
D1 = pd.DataFrame(D)
D1.to_csv(r"上海股票间的距离.csv")

In [9]:
def xiaoyong(x,singam,theta):
    lamda = np.zeros((len(x),6));
    lamda1 = np.ones((len(x),6));
    lamda2 = np.zeros((len(x),6));
    juhe = np.zeros((1,6));
    
    for i in range(0,len(x)): # 计算不同属性下的函数
        # 计算相对损失函数
        lamda[i,1] = singam * x[i];
        lamda[i,2] = x[i];
        lamda[i,3] = 1-x[i];
        lamda[i,4] = singam * (1-x[i]);
    
        # 计算基于后悔理论的效用函数
        lamda1[i,1] = 1 - (1 - np.exp(theta * lamda[i,1]))/theta
        lamda1[i,2] = 1 - (1 - np.exp(theta * lamda[i,2]))/theta
        lamda1[i,3] = 1 - (1 - np.exp(theta * lamda[i,3]))/theta
        lamda1[i,4] = 1 - (1 - np.exp(theta * lamda[i,4]))/theta
        
        # 计算相对效用函数
        lamda2[i,0] = 1 - lamda1[i,2] 
        lamda2[i,1] = lamda1[i,1] - lamda1[i,2] 
        lamda2[i,4] = lamda1[i,4] - lamda1[i,3] 
        lamda2[i,5] = 1 - lamda1[i,3]
        
    # 计算聚合相对效用函数(均值聚合)
    juhe = np.mean(lamda2,axis=0)
    
    return juhe

In [10]:
# 计算不同对象的聚合相对效用函数矩阵
singam = 0.15 
theta = 0.5
lamda = np.zeros((n,6));
for i in range(0,n):
    x = data2.iloc[i]
    lamda[i] = xiaoyong(x ,singam, theta)

#根据聚合相对效用函数计算不同对象的边界阈值
yuzhi = np.zeros((n,2))
for i in range(0,n):
    for j in range(0,len(lamda.T)):
        yuzhi[i][0] = (lamda[i][4])/(lamda[i][4]+lamda[i][0]-lamda[i][1])
        yuzhi[i][1] = (lamda[i][5]-lamda[i][4])/(lamda[i][1]+lamda[i][5]-lamda[i][4])
yuzhi = pd.DataFrame(yuzhi)
yuzhi.to_csv(r"上海股票的边界阈值.csv")

In [16]:
##根据归一化数据计算每个对象到正理想解和负理想解的距离
d1 = np.zeros((n,1)) 
d2 = np.zeros((n,1)) 
lishu = np.zeros((n,1))

for i in range(0,n):
    a = 0
    b = 0
    for j in range(0,m):
        a = a + (1 - data2.T[i][j])*(1 - data2.T[i][j])
        b = b + data2.T[i][j] * data2.T[i][j]
    d1[i] = np.sqrt(a) #每个对象到正理想解的距离
    d2[i] = np.sqrt(b) #每个对象到负理想解的距离
    lishu[i] = d1[i]/(d1[i]+d2[i])
lishuu = pd.DataFrame(lishu)
lishuu.to_csv(r"上海股票的隶属函数.csv")  

In [12]:
def qitayuzhi1(yz):
    #根据阈值设置下一个阈值的范围
    if yz[1] < 0.5:
        yuzhi2 = np.arange(yz[1],0.5,0.05) 
        V = []
        for k in range(0,len(yuzhi2)):
            yuzhi1 = 1-yuzhi2[k]
            x = 0 
            v1 = v2 = []
            v3 = 0
            
            #计算升高区域、降低区域、阴影区域的面积
            for i in range(0,n):
                if lishu[i] >= yuzhi1:
                    v1.append(1-lishu[i])
                elif lishu[i] <= yuzhi2[k]:
                    v2.append(lishu[i])
                else:
                    v3 = v3 + 1
            V1 = np.sum(v1)
            V2 = np.sum(v2)
            V3 = v3
    
            V.append(abs(V1+V2-V3))

    else:
        yuzhi2 = np.arange(yz[1],yz[0],0.05)
        x = 1
        V = []
        l = []
        
        for k in range(0,len(yuzhi2)):   
            yuzhi1 = np.arange(yuzhi2[k]+0.05,yz[0]+0.05,0.05)
            l.append(len(yuzhi1))
            for j in range(0,len(yuzhi1)):
                v1 = v2 = []
                v3 = 0
                
                #计算升高区域、降低区域、阴影区域的面积
                for i in range(0,n):
                    if lishu[i] >= yuzhi1[j]:
                        v1.append(1-lishu[i])
                    elif lishu[i] <= yuzhi2[k]:
                        v2.append(lishu[i])
                    else:
                        v3 = v3 + 1
                V1 = np.sum(v1)
                V2 = np.sum(v2)
                V3 = v3
    
                V.append(abs(V1+V2-V3))
        
    #根据最优化目标函数得到阈值
    if x == 0: 
        yz2 = yuzhi2[V.index(min(V))]
        yz1 = 1-yz2
    else:
        xx = V.index(min(V))
        for i in range(0,len(l)):
            if xx-l[i]>=0:
                xx = xx-l[i]
            else: 
                yz2 = yuzhi2[i]
                yuzhi1 = np.arange(yuzhi2[i]+0.05,yz[0]+0.05,0.05)
                yz1 = yuzhi1[xx]
    return yz1,yz2

In [13]:
df1 = pd.read_csv("上海股票的边界阈值.csv",index_col = 0)
df1.head()

,0,1
0,0.746400,0.064512
1,0.919965,0.211799
2,0.942318,0.277560
3,0.794213,0.083325
4,0.863115,0.129007


In [14]:
dis = pd.read_csv("上海股票间的距离.csv")
dis = dis.drop(['Unnamed: 0'],axis=1)
dis.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.000000,1.539524,0.936468,0.296016,0.051943,1.612173,0.580520,2.015450,0.522306,0.602965,...,0.000000,0.608588,1.379577,3.004449,1.087324,0.975738,2.521348,3.744021,1.294740,1.193096
1,1.539524,0.000000,0.419243,2.020658,1.258790,0.385186,1.396319,0.784665,0.692458,1.519171,...,1.132288,1.170424,0.055173,1.611153,0.071779,0.077907,0.700037,4.062523,0.060461,0.061165
2,0.936468,0.419243,0.000000,1.527018,0.819605,1.044873,1.146389,1.423871,0.228841,1.003335,...,0.708462,0.967768,0.612955,2.344143,0.513186,0.568880,1.021911,4.907117,0.642783,0.483550
3,0.296016,2.020658,1.527018,0.000000,0.374874,1.812683,0.014230,2.034515,1.016233,0.000000,...,0.477382,0.000000,1.692970,3.357814,1.392030,1.286460,3.047665,4.009225,1.604931,1.548691
4,0.051943,1.258790,0.819605,0.374874,0.000000,1.135899,0.375196,1.645257,0.439066,0.425857,...,0.041990,0.390807,1.143735,2.504830,1.039796,1.136054,2.116804,4.547584,1.176100,0.954096


In [15]:
y = 0
for l in range(0,n):
    sc = [];uc = []; dc =[]; 
    # 根据边界阈值进行三支决策
    for i in range(n):
        if dis.loc[l][i]>=df1.loc[l][0]:
            dc.append(i)
        elif dis.loc[l][i]<df1.loc[l][0] and dis.loc[l][i]>df1.loc[l][1]:
            uc.append(i)
        else:
            sc.append(i) 
        
    # 根据其他阈值继续进行三支决策
    yz = qitayuzhi1(df1.loc[l])
    #print(yz)
    while y == 0:
        yuzhi = yz
        for i in range(n):
            if i in uc:
                if dis.loc[l][i] <= yuzhi[1]:
                    sc.append(i)
                    uc.remove(i)
                if dis.loc[l][i] >= yuzhi[0]: 
                    dc.append(i)
                    uc.remove(i)
        if yuzhi[0]- 0.5 > 0.1 or len(uc) >= 1/4*n:
            yz = qitayuzhi1(yuzhi)
            # print(yz)
        else:
            y = 1
    print(l,'............')        
    print(sc)
    print(uc)
    print(dc)

0 ............
[0, 4, 17, 20, 3, 15, 16]
[8]
[1, 2, 5, 7, 10, 11, 12, 13, 14, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 6, 9, 21]
1 ............
[1, 10, 13, 18, 22, 24, 25, 28, 29]
[2, 5, 7, 8, 19, 26]
[0, 3, 4, 6, 9, 11, 12, 14, 15, 16, 17, 20, 21, 23, 27]
2 ............
[2, 8, 19]
[0, 1, 4, 10, 13, 17, 18, 20, 22, 24, 25, 28, 29]
[3, 5, 6, 7, 9, 11, 12, 14, 15, 16, 21, 23, 26, 27]
3 ............
[3, 6, 9, 15, 16, 21]
[0, 4, 17, 20]
[1, 2, 5, 7, 8, 10, 11, 12, 13, 14, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29]
4 ............
[0, 4, 17, 20]
[2, 3, 6, 8, 9, 15, 16, 19, 21]
[1, 5, 7, 10, 11, 12, 13, 14, 18, 22, 23, 24, 25, 26, 27, 28, 29]
5 ............
[5]
[1, 7, 10, 13, 18, 22, 24, 25, 26, 28, 29]
[0, 2, 3, 4, 6, 8, 9, 11, 12, 14, 15, 16, 17, 19, 20, 21, 23, 27]
6 ............
[3, 6, 9, 15, 16, 21]
[0, 4, 17, 20]
[1, 2, 5, 7, 8, 10, 11, 12, 13, 14, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29]
7 ............
[7, 26]
[1, 5, 10, 11, 13, 18, 22, 23, 24, 25, 28, 29]
[0, 2, 3, 4, 6, 8, 9, 12, 14, 15, 